In [ ]:
import pandas as pd
import numpy as np

In [ ]:
Data = pd.read_csv('House_Price_Regression.csv')

Data = Data.loc[:, ['latitude', 'longitude', 'price', 'size_in_m_2']]

Data.head()

In [ ]:
def k_medoids_Park_Jun(Data, random_seed, k, max_n_iter):

###########################################################################################

# Se generan los clusters iniciales aleatoreamente

    sample = resample(range(0, len(Data)), n_samples=len(Data), replace=False, random_state=random_seed)

    df_sample = pd.DataFrame({'index': range(0,len(Data)) , 'sample':sample})

    Q = []

    for q in np.arange(0 , 1 + 1/k , 1/k):

        Q.append( np.quantile( range(0, len(Data)) , q ).round(0) )



    labels_clusters = np.array([0])

    for j in range(0,k):
   
        labels_clusters = np.concatenate([labels_clusters, np.repeat( j, len( df_sample.loc[Q[j]:(math.floor(Q[j+1])-1), 'sample'] ) ) ] ) 
 

    df_cluster = pd.DataFrame({'observacion': df_sample['sample'] , 'cluster': labels_clusters})

###########################################################################################


# calculamos los medoids de los clusters iniciales

    M1 = Matrix_Dist_Euclidea(Data)

    M2 = M1 + M1.T

   ######################

    medoids = []

    for j in range(0, k):

      a = df_cluster.loc[df_cluster.cluster == j ,].observacion

      M3 = M2[a, :][: ,a] # matriz distancias para el cluster j

      suma_distancias = M3.sum(axis=1) # suma de la matriz por filas

      h = np.where( suma_distancias == min( suma_distancias ) ) # indice en la matriz de la observacion que minimiza la suma de distancias en el cluster j --> medoid del cluster j

      d = df_cluster.loc[df_cluster.cluster == j , ]    

      # d.iloc[h[0],:].observacion indice en el data set original de la observacion medoid del cluster j   

      medoids.append( Data[ d.iloc[h[0],:].observacion , : ]  ) # su componente j es el medoid del cluster j


###########################################################################################


   # Calculamos la suma de varianzas intra cluster para los clusters iniciales

    lista_suma_varianzas_intra_cluster = []

    distancias, varianzas_intra_cluster , suma_varianzas_intra_cluster = [], [], []

    for j in range(0,k):

        for i in df_cluster.loc[df_cluster.cluster == j, 'observacion']:

            distancias.append( Dist_Euclidea(Data[i,:], medoids[j]) )

        varianzas_intra_cluster.append( sum(distancias) )


    suma_varianzas_intra_cluster.append( sum(varianzas_intra_cluster) )

    lista_suma_varianzas_intra_cluster.append( suma_varianzas_intra_cluster )


###########################################################################################


# Asignar cada observacion a su cluster más cercano (paso 3-1 del articulo Park y Jun)

# En el articulo de Park y Jun no se especifica como hacer esto.

# Se me ocurren dos opciones. Una es la expuesta en mi algoritmo de k-means (pasos 4, 5 y 6). 

# La otra opción es la que creo que está más cerca de lo que Park y Jun quieren expresar:
# 1) Se calculan las distancias entre x_i y cada uno de los medoids de los clusters existentes, 
#    Se asigna x_i al cluster cuyo medoid minimiza esas distancias.
# 2) Con ello se obtiene una nueva configuracion de clusters. Se calcula la suma de varianzas intra cluster de esa nueva configuracion.
# Si la suma de varianzas en igual a la anterior, se para. Si es menor, se repite lo anterior (pasos 1) y 2))


    n_iter = []

    iter_condicion_cluster_vacio = []


    for r in range(0 , max_n_iter):

        n_iter.append(r)
       

       # Calculamos los nuevos clusters 

        labels_clusters = []

        for i in df_sample['sample']:

            distancias = []
    
            for j in range(0,k):

                distancias.append( Dist_Euclidea(Data[i,:], medoids[j]) )

                

            df_distancias = pd.DataFrame({'Distancias' : distancias , 'Cluster': range(0,k)})

            df_distancias_sort = df_distancias.sort_values(by='Distancias', ascending=True)

            labels_clusters.append( df_distancias_sort.iloc[0]['Cluster'] )

        
        
        df_cluster = pd.DataFrame({'observacion' : df_sample['sample'] , 'cluster': labels_clusters})



        # Puede ser que se llegue a una configuracion de clusters en la que algun cluster no tiene observaciones
        # En ese caso vamos a considerar dos respuestas. O bien se deja que el algoritmo haga una seleccion de k. 
        # O bien se fuerza a que k sea el fijado, aignando una observacion aleatoriamente a ese cluster vacio.
        # Vamos a implementar la segunda.

        #for j in range(0,k):

            #if len(df_cluster.loc[df_cluster.cluster == j ,]) == 0 :

                #iter_condicion_cluster_vacio.append(r)

                # Si un cluster esta vacio (no se le ha asignado ningun elemento), se le asigna una observacion al azar
                # Y se elimina esa observacion del cluster al que pertenecia.

                #m = resample(range(0, len(Data)), n_samples=1 , replace=False)

                # Eliminamos la observacion m del cluster al que se le habia asignado 

                #df_cluster = df_cluster.drop(df_cluster.loc[df_cluster.observacion == m[0] , :].index)

                # Asignamos la observacion m al cluster vacio

                #df_cluster = pd.concat([df_cluster , pd.Series({'observacion': df_sample.loc[m[0] , 'sample'], 'cluster': j }).to_frame().T ], ignore_index=True)
                



###########################################################################################


       # Calculamos los medoids de los nuevos clusters

        medoids = []
        label_medoids =[]


        for j in range(0, k):                    
                
                a = df_cluster.loc[df_cluster.cluster == j ,].observacion

                M3 = M2[a, :][: ,a] # matriz distancias para el cluster j

                suma_distancias = M3.sum(axis=1) # suma de la matriz por filas

                h = np.where( suma_distancias == min( suma_distancias) ) # indice en la matriz de la observacion que minimiza la suma de distancias en el cluster j --> medoid del cluster j

                d = df_cluster.loc[df_cluster.cluster == j , ]    

                # d.iloc[h[0],:].observacion indice en el data set original de la observacion medoid del cluster j   

                medoids.append( Data[ d.iloc[h[0],:].observacion , : ]  ) # su componente j es el medoid del cluster j
  
                label_medoids.append(d.iloc[h[0],:].observacion)



###########################################################################################

        # Calculamos la suma de varianzas intra cluster para los nuevos clusters

        distancias, varianzas_intra_cluster , suma_varianzas_intra_cluster = [], [], []

        for j in range(0,k):

            for i in df_cluster.loc[df_cluster.cluster == j, 'observacion']:

                distancias.append( Dist_Euclidea(Data[i,:], medoids[j]) )

            varianzas_intra_cluster.append( sum(distancias) )


        suma_varianzas_intra_cluster.append( sum(varianzas_intra_cluster) )

        lista_suma_varianzas_intra_cluster.append( suma_varianzas_intra_cluster )


        #####################

        # if lista_suma_varianzas_intra_cluster[r+1] != lista_suma_varianzas_intra_cluster[r] :

            # continue

        # else :

            # break


        ###########################################

    df_cluster_final = pd.DataFrame({'observaciones': df_sample['sample']  , 'cluster': labels_clusters })



    return df_cluster_final, df_cluster, lista_suma_varianzas_intra_cluster, n_iter, iter_condicion_cluster_vacio, label_medoids, medoids
